In [2]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# Google Drive에 저장한 json 파일 불러오기
import json
from datasets import Dataset
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/sample_data/travel_chatbot_data.json'

# 데이터셋 로드
with open(file_path, 'r') as file:
    data = json.load(file)

# Dataset 생성
train_data = [{"text": f"{row['prompt']}\n{row['response']}"} for row in data]
dataset = Dataset.from_dict({"text": [item["text"] for item in train_data]})

Mounted at /content/drive


In [5]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# 모델과 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# 1. pad_token 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # eos_token을 pad_token으로 설정
    model.resize_token_embeddings(len(tokenizer))  # 모델의 임베딩 크기 재조정

# 데이터 전처리 함수
def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",  # Padding을 사용
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()  # labels 추가
    return tokenized

# 데이터 전처리
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Fine-tuning 설정
training_args = TrainingArguments(
    output_dir="./kogpt2-results",
    evaluation_strategy="no",  # 평가 비활성화
    report_to="none",  # wandb 비활성화
    learning_rate=5e-5,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=200,
    save_total_limit=2,
    logging_dir="./logs"
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# 모델 학습
trainer.train()

# 모델 저장
model.save_pretrained("./fine_tuned_kogpt2")
tokenizer.save_pretrained("./fine_tuned_kogpt2")


Map:   0%|          | 0/732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-d9adbd2859f7>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


('./fine_tuned_kogpt2/tokenizer_config.json',
 './fine_tuned_kogpt2/special_tokens_map.json',
 './fine_tuned_kogpt2/vocab.json',
 './fine_tuned_kogpt2/merges.txt',
 './fine_tuned_kogpt2/added_tokens.json',
 './fine_tuned_kogpt2/tokenizer.json')

In [17]:
from transformers import pipeline

# Fine-tuned 모델 로드
model_name = "./fine_tuned_kogpt2"
qa_pipeline = pipeline("text-generation", model=model_name, tokenizer=model_name)

# 질문 생성
question = "제주도에서 갈만한 여행지 1곳만 추천해줘.\n"
# 텍스트 생성
result = qa_pipeline(
    question,
    max_length=50,  # 답변의 최대 길이 제한
    num_return_sequences=1,  # 하나의 답변만 생성
    top_p=0.9,  # 확률 누적 분포 제한
    temperature=0.7,  # 무작위성 조정
    repetition_penalty=1.2  # 반복 패널티 추가
)

# 생성된 텍스트 후처리
generated_text = result[0]["generated_text"]

# 온점 기준으로 텍스트 잘라서 마지막 문장 제거
if "." in generated_text:
    processed_text = generated_text[:generated_text.rfind(".") + 1]
else:
    processed_text = generated_text  # 온점이 없으면 원본 텍스트 유지

print(processed_text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


제주도에서 갈만한 여행지 1곳만 추천해줘.
용산신학교 - 용산 신학교는 일제강점기 징용과 수탈의 현장이자 해방 이후 한국 기독교계의 중심축이었던 양 교단의 역사를 가진 학교입니다.
